# Derivative of traits_i with respect to traits_i

Here I will be testing my solution for
$\frac{ \partial \mathbf{\hat{V}_i} }{ \partial \mathbf{V_i} }$
(see below)
by calculating the Jacobian using the `theano` package
and comparing those results to my solution.




## Importing packages and setting options

In [33]:
%env OMP_NUM_THREADS=4
%env THEANO_FLAGS='openmp=True'
import sympy
import theano
theano.config.cxx = ""
import theano.tensor as T
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
pd.options.display.max_columns = 10

env: OMP_NUM_THREADS=4
env: THEANO_FLAGS='openmp=True'


## Equations

__Notes:__

- $*$ is matrix multiplication.
- ${}^\text{T}$ represents transpose.
- Elements in __bold__ are matrices
- $\mathbf{I}$ is an identity matrix with the same number of 
  rows and columns as the number of traits in $\mathbf{V_i}$
- Run `vignette("model", "sauron")` in R to see more of the model and
  what each parameter means.
  

The equations for (1) traits for species $i$ at time $t+1$ ($\mathbf{\hat{V}_i}$)
and (2) the partial derivative of species $i$ traits with respect
to species $k$ traits (where $k \ne i$)
are as follows:

\begin{align}
\mathbf{\hat{V}_i} &= \mathbf{V_i} + \sigma^2
    \left[
        2 ~ g ~ Z ~ \text{e}^{-\mathbf{V_i} * \mathbf{V_i}^\text{T}} \mathbf{V_i} 
        - f ~ \mathbf{V_i} * (\mathbf{C} + \mathbf{C}^\text{T})
    \right] \\
    Z &= N_i + \sum_{j \ne i}^{n}{ N_j ~ \text{e}^{-d \mathbf{V_j} * \mathbf{V_j}^\text{T}} } \\
    \frac{ \partial \mathbf{\hat{V}_i} }{ \partial \mathbf{V_i} } &= \mathbf{I} + \sigma^2 ~
        \left[
            2 ~ g ~ Z ~ \text{e}^{ -\mathbf{V_i} * \mathbf{V_i}^{\text{T}} }
            \left(
                -2 ~ \mathbf{V_i}^{\text{T}} * \mathbf{V_i} + \mathbf{I}
            \right) -
            f \left( \mathbf{C} + \mathbf{C}^\text{T} \right)
        \right]
\end{align}


## Read CSV of simulated datasets

In [35]:
sims = pd.read_csv("simulated_data.csv")
sims.head()

,V1,V2,V3,V4,V5,...,f,g,eta,r0,d
0,5.329784,-0.593159,0.003065,1.414273,-6.458124,...,0.137235,0.104261,0.063997,0.343463,-0.118705
1,-1.514917,-1.024847,5.413096,-4.548136,1.542865,...,0.600063,0.197839,0.103529,0.279827,-0.158496
2,-9.969353,0.930724,2.855755,8.144096,3.640262,...,0.537799,0.202685,-0.088763,0.303346,-0.159742
3,3.821274,-3.732219,-2.680385,-1.586652,-9.755770,...,0.123312,0.117315,-0.082240,0.136664,0.103837
4,3.291826,0.708288,-5.281580,6.224788,-0.271641,...,0.560044,0.054967,0.046302,0.254523,-0.125201


## Functions to compare methods

In [36]:
def automatic(i, N, V, Z, CCC, f, g, s2):
    """Automatic differentiation using theano pkg"""
    Vi = T.dvector('Vi')
    Vhat = Vi + s2 * (
        ( 2 * g * Z * T.exp(-1 * T.dot(Vi, Vi.T)) * Vi) - 
        ( f * T.dot(Vi, CCC) )
    )
    J, updates = theano.scan(lambda i, Vhat, Vi : T.grad(Vhat[i], Vi), 
                         sequences=T.arange(Vhat.shape[0]), non_sequences=[Vhat, Vi])
    num_fun = theano.function([Vi], J, updates=updates)
    out_array = num_fun(V[i,:]).T
    return out_array

In [37]:
def symbolic(i, V, Z, CCC, f, g, s2):
    """Symbolic differentiation using my brain"""
    q = V.shape[1]
    I = np.identity(q)
    Vi = V[i,:]
    Vi = Vi.reshape((1, q))
    dVhat = I + s2 * (
        ( 2 * g * Z * np.exp(-1 * Vi @ Vi.T)[0,0] * (I - 2 * Vi.T @ Vi) ) -
        (f * CCC)
    )
    return dVhat

In [38]:
def compare_methods(sim_i, s2 = 0.01, abs = False):
    """Compare answers from symbolic and automatic methods"""
    
    # Fill info from data frame:
    N = sims.loc[sim_i, [x.startswith("N") for x in sims.columns]].values
    V = sims.loc[sim_i, [x.startswith("V") for x in sims.columns]].values
    n, q = (N.size, int(V.size / N.size))
    V = V.reshape((n, q), order = 'F')
    f = sims.loc[sim_i,"f"]
    g = sims.loc[sim_i,"g"]
    eta = sims.loc[sim_i,"eta"]
    d = sims.loc[sim_i,"d"]
    C = np.zeros((q, q)) + eta
    np.fill_diagonal(C,1.0)
    CCC = C + C.T
    
    # Create output array:
    diffs = np.empty((n, 4))
    diffs[:,0] = sim_i
    
    # Fill output array:
    for i in range(0, n):
        Z = N[i] + np.sum([np.exp(-d * np.dot(V[j,:], V[j,:].T)) * N[j] 
            for j in range(0, N.size) if j != i])
        auto = automatic(i, N, V, Z, CCC, f, g, s2)
        sym = symbolic(i, V, Z, CCC, f, g, s2)
        if abs:
            diff = auto - sym
        else:
            diff = (auto - sym) / sym
        diff = diff.flatten()
        diffs[i, 1] = i
        diffs[i, 2] = diff.min()
        diffs[i, 3] = diff.max()
    
    return diffs

### Example of using `compare_methods`:

In [39]:
diffs = compare_methods(0)
# Worst case examples:
print(diffs[:,2].min())
print(diffs[:,3].max())

-4.2579648295609155e-16
0.0


## Comparing methods

This takes ~2 minutes.

In [28]:
n_per_rep = 4
diffs = np.empty((int(n_per_rep * 100), 4))

In [29]:
for rep in tqdm(range(100)):
    diffs_r = compare_methods(rep)
    diffs[(rep * n_per_rep):((rep+1) * n_per_rep),:] = diffs_r

100%|██████████| 100/100 [02:11<00:00,  1.37s/it]


## The results
They appear to be extremely similar, enough so that I feel comfortable with my symbolic version.

In [30]:
print(diffs[:,2].min())
print(diffs[:,3].max())

-4.7219454061666434e-14
1.752071503448691e-14


## Write output to file

To make sure the R version works, too, I'm writing to a CSV file the output from the symbolic version on the 100 datasets.

In [42]:
n = np.sum([x.startswith("N") for x in sims.columns])
q = int(np.sum([x.startswith("V") for x in sims.columns]) / n)
s2 = 0.01
# Output array
results = np.zeros((100, n * q * q))

for sim_i in range(100):
    
    # Fill info from data frame:
    N = sims.loc[sim_i, [x.startswith("N") for x in sims.columns]].values
    V = sims.loc[sim_i, [x.startswith("V") for x in sims.columns]].values
    V = V.reshape((n, q), order = 'F')
    f = sims.loc[sim_i,"f"]
    g = sims.loc[sim_i,"g"]
    eta = sims.loc[sim_i,"eta"]
    d = sims.loc[sim_i,"d"]
    C = np.zeros((q, q)) + eta
    np.fill_diagonal(C,1.0)
    CCC = C + C.T

    # Fill output array:
    for i in range(0, n):
        Z = N[i] + np.sum([np.exp(-d * np.dot(V[j,:], V[j,:].T)) * N[j] 
            for j in range(0, N.size) if j != i])
        sym = symbolic(i, V, Z, CCC, f, g, s2)
        results[sim_i, (i*q*q):((i+1)*q*q)] = sym.flatten()

# Make sure first and last aren't zeros:
results[[0, 99], :]

array([[ 9.97255309e-01, -1.75653056e-04, -1.75653056e-04,
        -1.75653056e-04,  9.97255309e-01, -1.75653056e-04,
        -1.75653056e-04, -1.75653056e-04,  9.97255309e-01,
         9.97255309e-01, -1.75653056e-04, -1.75653056e-04,
        -1.75653056e-04,  9.97255309e-01, -1.75653056e-04,
        -1.75653056e-04, -1.75653056e-04,  9.97255309e-01,
         1.60663243e+00,  9.27146520e-03, -1.20901663e-03,
         9.27146520e-03, -6.18695473e+00,  8.52319329e-01,
        -1.20901663e-03,  8.52319329e-01,  1.51339457e+00,
         9.97251834e-01, -1.88198005e-04, -1.81311180e-04,
        -1.88198005e-04,  9.97222504e-01, -1.90971899e-04,
        -1.81311180e-04, -1.90971899e-04,  9.97249559e-01],
       [ 9.94402392e-01,  3.81319643e-04,  3.81319643e-04,
         3.81319643e-04,  9.94402392e-01,  3.81319643e-04,
         3.81319643e-04,  3.81319643e-04,  9.94402392e-01,
         9.94402392e-01,  3.81319643e-04,  3.81319643e-04,
         3.81319643e-04,  9.94402392e-01,  3.81319643e-

In [43]:
np.savetxt('results/dVi_dVi.csv', results, delimiter=',')